# Deep Agents

[deepagents](https://docs.langchain.com/oss/python/deepagents/overview) 可以看作是 LangChain 团队出品的 DeepResearch。本节通过查询一个近期新闻，检验 Deep Agents 是否有主动调用搜索、深度洞察的能力。

> 这里只是简单 quickstart 一下，更多信息大家去官网看吧！

In [1]:
# !pip install deepagents dashscope

**1）加载模型**

In [2]:
import os
import dashscope

from datetime import datetime
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from deepagents import create_deep_agent
from dashscope import Generation

# 加载模型配置
_ = load_dotenv()

# 为灵积配置 api_key
dashscope.api_key=os.getenv("DASHSCOPE_API_KEY")

# 加载模型
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    model="qwen3-max",
)

**2）创建搜索工具**

我们使用上一节创建的 `dashscope_search` 搜索工具。

In [3]:
@tool
def dashscope_search(query: str) -> str:
    """使用夸克搜索 API 搜索互联网信息"""
    response = Generation.call(
        model='qwen-max',
        prompt=query,
        enable_search=True,
        result_format='message'
    )

    if response.status_code == 200:
        return response.output.choices[0].message.content
    else:
        return (
            "Search failed with status code: "
            f"{response.status_code}, message: {response.message}"
        )

**3）创建深度代理**

In [4]:
# System prompt to steer the agent to be an expert researcher
research_instructions = """你是一名资深研究员。你的工作是进行全面深入的研究，并撰写一份精炼的报告。

你可以使用互联网搜索工具作为获取信息的主要方式。

## `dashscope_search`

使用该工具对指定查询进行互联网搜索。
"""

@tool
def get_today_date() -> str:
    """获取今天的日期"""
    return datetime.now().strftime("%Y-%m-%d")

agent = create_deep_agent(
    model=llm,
    tools=[dashscope_search, get_today_date],
    system_prompt=research_instructions
)

**4）运行 Agent**

我们的问题是：

```
新上任的玻利维亚总统是谁？请介绍一下这位总统。
```

因为近期（2025 年 11 月 8 日）玻利维亚总统 `罗德里戈・帕斯・佩雷拉` 刚刚上任，这条信息肯定不在预训练数据里。可以用这个问题验证 Deep Agents 是否真的联网了。

In [5]:
result = agent.invoke({"messages": [
    {"role": "user", "content": "新上任的玻利维亚总统是谁？请介绍一下这位总统。"}
]})

# 最终回复
print(result["messages"][-1].content)

# 玻利维亚新任总统：罗德里戈·帕斯·佩雷拉

## 基本信息
罗德里戈·帕斯·佩雷拉（Rodrigo Paz Pereira）是玻利维亚现任总统，于2025年11月8日在行政首都拉巴斯宣誓就职，开始为期5年的总统任期。他出生于1967年9月22日，在西班牙的圣地亚哥德孔波斯特拉出生，并在玻利维亚拉巴斯市完成学业。他拥有美国美利坚大学的政治管理硕士学位。

## 政治背景
帕斯·佩雷拉来自基督教民主党，在当选总统前曾担任塔里哈市市长及塔里哈省参议员。他的当选结束了左翼政党"争取社会主义运动"（MAS）自2006年以来近20年的执政历史，标志着玻利维亚政治格局的重大转变。

## 主要政策主张
1. **经济政策**：
   - 重新分配中央财政资源至地方，促进地方发展
   - 推行普惠信贷和税收减免刺激经济增长
   - 开放商品进口壁垒，特别是对玻利维亚无法生产的商品
   - 提出"资本主义惠及所有人，而非少数人"的经济发展理念

2. **能源与资源政策**：
   - 在以化石燃料为主的经济体中推行清洁能源政策
   - 推动锂矿资源国有化战略，加强国家对锂矿资源的控制权
   - 同时吸引外资开发锂矿与天然气资源

3. **社会政策**：
   - 维持社会福利项目，确保弱势群体得到保障
   - 强调打击腐败行为，增强政府透明度

4. **外交政策**：
   - 与美国全面恢复外交关系，结束了两国中断17年的大使级外交关系
   - 旨在通过改善外交关系提振经济、吸引外资

## 面临的挑战
作为一位温和务实的领导人，帕斯·佩雷拉面临着多重挑战：
- 改善经济状况
- 降低通货膨胀率
- 解决外汇短缺问题
- 平衡国内政治力量，避免加剧社会紧张局势

在他的领导下，玻利维亚正尝试走出一条结合开放与改革的新发展道路。


In [6]:
# 思考过程
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

新上任的玻利维亚总统是谁？请介绍一下这位总统。
================================== Ai Message ==================================

我需要查找新上任的玻利维亚总统的信息。让我通过互联网搜索来获取最新信息。
Tool Calls:
  dashscope_search (call_705df22f455646c9aba7f011)
 Call ID: call_705df22f455646c9aba7f011
  Args:
    query: 新上任的玻利维亚总统是谁
================================= Tool Message =================================
Name: dashscope_search

新上任的玻利维亚总统是罗德里戈·帕斯·佩雷拉。他于2025年11月8日在玻利维亚行政首都拉巴斯宣誓就职，开始了为期5年的任期。帕斯在总统选举中胜出，并且宣布与美国全面恢复外交关系。
================================== Ai Message ==================================

我需要获取更多关于罗德里戈·帕斯·佩雷拉的详细信息，以便全面介绍这位新上任的玻利维亚总统。
Tool Calls:
  dashscope_search (call_38db699b37944a5daeb9fd4a)
 Call ID: call_38db699b37944a5daeb9fd4a
  Args:
    query: 罗德里戈·帕斯·佩雷拉 玻利维亚总统 背景 政策
================================= Tool Message =================================
Name: dashscope_search

罗德里戈·帕斯·佩雷拉，玻利维亚现任总统，出生于1967年9月22日，在西班牙的圣地亚哥德孔波斯特拉出生，并在玻利维亚的拉巴斯市